In [1]:
import pandas as pd
import numpy as np
import os

# 1. 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 기준 파일 로드
sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

# 3. 모델 로드 및 정렬 함수
def load_and_align(filename):
    file_path = os.path.join(SUB_PATH, filename)
    df = pd.read_csv(file_path)
    
    if len(df) != len(sample_sub):
        raise ValueError(f"[Error] {filename} 행 개수 불일치!")
    
    # 인덱스 정렬 안전장치
    if 'index' in df.columns:
         df = df.set_index('index').reindex(sample_sub['index']).reset_index()
    elif 'id' in df.columns:
         df = df.set_index('id').reindex(sample_sub['id']).reset_index()
         
    return df['voted'].values

print(">>> 데이터 로딩 중...")
# [기존 3대장]
s1_voted = load_and_align('0130-1923.csv')               # m1 (Baseline)
s2_voted = load_and_align('Pure_0781_AUC_0.77322.csv')   # m2 (AUC Optimized)
s7_voted = load_and_align('m7_Refined_test_preds.csv')   # m7 (Deep FE)

# [New Challenger: m4]
target_file_m4 = 'Diversity_Focal_SNN_AUC_0.7708.csv'    # m4 (SNN)
s4_voted = load_and_align(target_file_m4)

# --------------------------------------------------------------------------------
# [Smart Direction Check]
# --------------------------------------------------------------------------------
print("-" * 30)
models = {'m2': s2_voted, 'm7': s7_voted, 'm4': s4_voted}
model_flags = {}

for name, preds in models.items():
    corr = np.corrcoef(s1_voted, preds)[0, 1]
    print(f"Checking Correlation (m1 vs {name}): {corr:.4f}")
    if corr < 0:
        print(f"!!! 경고: {name} 역방향 감지 -> 자동 반전 설정 !!!")
        model_flags[name] = False
    else:
        model_flags[name] = True

# 4. 4-Way Rank Ensemble (Optimized Mix)
print("-" * 30)
print(">>> 4-Way 랭크 앙상블 계산 중...")

r1 = pd.Series(s1_voted).rank(pct=True)

def get_rank(preds, is_ascending):
    series = pd.Series(preds)
    return series.rank(pct=True) if is_ascending else (1.0 - series.rank(pct=True))

r2 = get_rank(s2_voted, model_flags['m2'])
r7 = get_rank(s7_voted, model_flags['m7'])
r4 = get_rank(s4_voted, model_flags['m4'])

# [최적 가중치]
# m1(20) + m2(20): 안정성 (Base)
# m7(45): 성능 (Main Engine) - 60%는 과함, 45%가 적절
# m4(15): 변수 (Booster)
avg_rank = (r1 * 0.20) + (r2 * 0.20) + (r7 * 0.45) + (r4 * 0.15)

# 5. Scale Restoration (m1 분포 복원)
print(">>> m1 분포로 값 매핑(Restoration) 중...")
m1_values_sorted = np.sort(s1_voted)
final_int_rank = avg_rank.rank(method='first').astype(int).values - 1
final_voted = m1_values_sorted[final_int_rank]

# 6. 저장
submission = sample_sub.copy()
submission['voted'] = final_voted

output_name = os.path.join(SUB_PATH, '46_QUARTET_OPTIMIZED_20_20_45_15.csv')
submission.to_csv(output_name, index=False)

print("-" * 50)
print(f"파일 생성 완료: {output_name}")
print("전략: m1(20) + m2(20) + m7(45) + m4(15)")
print("예측: Heavy m7(2:2:6)의 과적합 리스크를 줄이고, m4의 다양성을 더해 0.7816+ 목표")
print("-" * 50)

>>> 데이터 로딩 중...
------------------------------
Checking Correlation (m1 vs m2): 0.9997
Checking Correlation (m1 vs m7): -0.9910
!!! 경고: m7 역방향 감지 -> 자동 반전 설정 !!!
Checking Correlation (m1 vs m4): 0.9751
------------------------------
>>> 4-Way 랭크 앙상블 계산 중...
>>> m1 분포로 값 매핑(Restoration) 중...
--------------------------------------------------
파일 생성 완료: ./submissions/46_QUARTET_OPTIMIZED_20_20_45_15.csv
전략: m1(20) + m2(20) + m7(45) + m4(15)
예측: Heavy m7(2:2:6)의 과적합 리스크를 줄이고, m4의 다양성을 더해 0.7816+ 목표
--------------------------------------------------
